In [0]:
external_azure_path = 'abfss://container20260122@datalake20260121.dfs.core.windows.net/'

dbutils.fs.ls(f'{external_azure_path}/01-bronze')

In [0]:
# Creating tmp tables to load product data into silver layer
allowed_tables = frozenset({'orders/', 'stores/', 'staffs/', 'order_items/'})

for delta_table in dbutils.fs.ls(f'{external_azure_path}/01-bronze'):
    if delta_table.name in allowed_tables:
        (
            spark.read
                .format("delta")
                .load(delta_table.path)
                .createOrReplaceTempView(f'tmp_bronze_{delta_table.name.rstrip('/')}')
        )

In [0]:
%sql
DESCRIBE tmp_bronze_orders 

In [0]:
%py
df_silver_orders = spark.sql('''
    -- Creating SQL query to aggregate order data
    WITH cte_order_items AS (
      SELECT
          order_id
        , item_id
        , product_id
        , quantity
        , list_price
        , ROUND((list_price * quantity) * (1 - discount), 2) AS total_sale
        , discount
      FROM 
        tmp_bronze_order_items
    )
    
    SELECT
        od.order_id
      , od.customer_id
      , CASE
          WHEN od.order_status = 1 THEN 'Pending'
          WHEN od.order_status = 2 THEN 'Processing'
          WHEN od.order_status = 3 THEN 'Shipped'
          WHEN od.order_status = 4 THEN 'Delivered'
          ELSE 'Unknown'
        END               AS status
      , od.order_status   AS status_id
      , od.order_date
      , od.required_date  AS required_date
      , od.shipped_date   AS shipped_date
      , st.store_name
      , st.state          AS store_state
      , st.city           AS store_city
      , sf.first_name     AS staff_first_name
      , sf.active         AS staff_is_active
      , sf.email          AS staff_email
      , oi.product_id 
      , oi.quantity       AS item_quantity
      , oi.total_sale     AS total_sales
      , oi.list_price     AS item_list_price
      , oi.discount       AS discount
    FROM
      tmp_bronze_orders           AS od
      LEFT JOIN tmp_bronze_stores AS st ON od.store_id  = st.store_id
      LEFT JOIN tmp_bronze_staffs AS sf ON od.staff_id  = sf.staff_id
      LEFT JOIN cte_order_items   AS oi ON od.order_id  = oi.order_id
''')

display(df_silver_orders)

In [0]:
# Writing on silver azure external storage in delta format
(df_silver_orders.write
    .mode('overwrite')
    .format('delta')
    .option('mergeSchema', 'true')
    .save(f'{external_azure_path}/02-silver/orders')
)

In [0]:
%sql
-- Creating a new schema to store silver tables
CREATE SCHEMA IF NOT EXISTS bike_store.silver_schema

In [0]:
%sql
DROP TABLE IF EXISTS bike_store.silver_schema.orders

In [0]:
%sql
-- Creating databricks delta table
CREATE TABLE IF NOT EXISTS bike_store.silver_schema.orders
LOCATION 'abfss://container20260122@datalake20260121.dfs.core.windows.net/02-silver/orders'